<a href="https://colab.research.google.com/github/richardlin01/CodeReview/blob/main/GiaiThich.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Khai báo các thư viện

In [ ]:
#include <FirebaseESP32.h>
#include <HardwareSerial.h>
#include <WiFi.h>